In [5]:
import pytest
import numpy as np
import pandas as pd
import torch
from sklearn.linear_model import LogisticRegression
from src.covmetrics.ERT import ERT
from unittest.mock import patch


def dummy_loss(pred, y):
    return (pred - y)**2

def alt_loss(pred, y):
    return (pred - y)**2

In [6]:
@pytest.mark.parametrize("backend", ["numpy", "torch", "dataframe"])
def test_evaluate_all_basic(backend):
    n_samples, n_features = 20, 5
    if backend == "numpy":
        X = np.random.rand(n_samples, n_features)
        cover = np.random.randint(0, 2, size=n_samples)
    elif backend == "torch":
        X = torch.rand(n_samples, n_features)
        cover = torch.randint(0, 2, size=(n_samples,))
    elif backend == "dataframe":
        X = pd.DataFrame(np.random.rand(n_samples, n_features), columns=[f"f{i}" for i in range(n_features)])
        cover = pd.Series(np.random.randint(0, 2, size=n_samples))

    ert = ERT(LogisticRegression, max_iter=100)
    ert.fit(X, cover)

    with patch("src.covmetrics.ERT.check_tabular"), \
         patch("src.covmetrics.ERT.check_cover"), \
         patch("src.covmetrics.ERT.check_consistency"), \
         patch("src.covmetrics.ERT.check_alpha"), \
         patch.object(ERT, "make_losses"), \
         patch.object(ERT, "init_model"):

        results = ert.evaluate_all(X, cover, alpha=0.8, n_splits=None)
        assert isinstance(results, dict)
        for key, value in results.items():
            assert isinstance(value, float)


@pytest.mark.parametrize("backend", ["numpy", "torch", "dataframe"])
def test_evaluate_all_with_cv(backend):
    n_samples, n_features = 20, 5
    if backend == "numpy":
        X = np.random.rand(n_samples, n_features)
        cover = np.random.randint(0, 2, size=n_samples)
    elif backend == "torch":
        X = torch.rand(n_samples, n_features)
        cover = torch.randint(0, 2, size=(n_samples,))
    elif backend == "dataframe":
        X = pd.DataFrame(np.random.rand(n_samples, n_features), columns=[f"f{i}" for i in range(n_features)])
        cover = pd.Series(np.random.randint(0, 2, size=n_samples))

    ert = ERT(LogisticRegression, max_iter=100)
    ert.fit(X, cover)

    with patch("src.covmetrics.ERT.check_tabular"), \
         patch("src.covmetrics.ERT.check_cover"), \
         patch("src.covmetrics.ERT.check_consistency"), \
         patch("src.covmetrics.ERT.check_alpha"), \
         patch.object(ERT, "make_losses"), \
         patch.object(ERT, "init_model"):

        results = ert.evaluate_all(X, cover, alpha=0.7, n_splits=3)
        assert isinstance(results, dict)
        for key, value in results.items():
            assert isinstance(value, float)


def test_evaluate_all_exception_if_not_fitted():
    X = np.random.rand(10, 5)
    cover = np.random.randint(0, 2, size=10)
    ert = ERT(LogisticRegression, max_iter=100)
    with patch("src.covmetrics.ERT.check_tabular"), \
         patch("src.covmetrics.ERT.check_cover"), \
         patch("src.covmetrics.ERT.check_consistency"), \
         patch("src.covmetrics.ERT.check_alpha"), \
         patch.object(ERT, "make_losses"):
        with pytest.raises(Exception):
            ert.evaluate_all(X, cover, alpha=0.8, n_splits=None)


@pytest.mark.parametrize("backend", ["numpy", "torch", "dataframe"])
def test_evaluate_all_with_under_over_confidence(backend):
    n_samples, n_features = 15, 4
    if backend == "numpy":
        X = np.random.rand(n_samples, n_features)
        cover = np.random.randint(0, 2, size=n_samples)
    elif backend == "torch":
        X = torch.rand(n_samples, n_features)
        cover = torch.randint(0, 2, size=(n_samples,))
    elif backend == "dataframe":
        X = pd.DataFrame(np.random.rand(n_samples, n_features))
        cover = pd.Series(np.random.randint(0, 2, size=n_samples))

    ert = ERT(LogisticRegression)
    ert.fit(X, cover)

    with patch("src.covmetrics.ERT.check_tabular"), \
         patch("src.covmetrics.ERT.check_cover"), \
         patch("src.covmetrics.ERT.check_consistency"), \
         patch("src.covmetrics.ERT.check_alpha"), \
         patch.object(ERT, "make_losses"), \
         patch.object(ERT, "init_model"):

        results = ert.evaluate_all(X, cover, alpha=0.6, n_splits=None, underconfidence=True, overconfidence=True)
        assert isinstance(results, dict)
        for key, value in results.items():
            assert isinstance(value, float)


In [7]:
test_evaluate_all_basic("numpy")

UnboundLocalError: cannot access local variable 'results' where it is not associated with a value